In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.callbacks import History, EarlyStopping, Callback
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout

from scipy import signal
from scipy.fft import fft, rfft
from statistics import stdev 
import ast

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier



import more_itertools as mit

In [2]:
#l_w = 200# Window length
l_ws = [250] 
n_p = 1 # Number of predictions
#npersegFFT = 30 #samples per second FFT
nfft = [10,20,30]

In [3]:
def shape_data(arr,train=True):
    data = []
    for i in range(len(arr) - l_w - n_p):
        data.append(arr[i:i + l_w + n_p])
    data = np.array(data)

    assert len(data.shape) == 3
    if train:
        X_train = data[:, :-n_p, :]
        y_train = data[:, -n_p:, 0]  # telemetry value is at position 0
        
        return X_train,y_train
    else:
        X_test = data[:, :-n_p, :]
        y_test = data[:, -n_p:, 0]  # telemetry value is at position 0
        return X_test,y_test
    
def ema(values, period):
    values = np.array(values)
    return pd.ewma(values, span=period)[-1]

In [4]:
## LISTA DE ANOMALÍAS
path = ''
df = pd.read_csv(path+'anom_types_.csv')
df.head()

,chan_id,spacecraft,anomaly_sequences,class,num_values,type,class1,class2,Class_Or,L,Unnamed: 10,MA,Unnamed: 12,F,Unnamed: 14
0,A-1,SMAP,"[[4690, 4774]]",[point],8640,[binary],point,point,point,1,1,1,0,1,0
1,A-2,SMAP,"[[4450, 4560]]",[contextual],7914,"[continuous,periodic]",wave,frequency,wave,1,0,1,1,1,1
2,A-3,SMAP,"[[4575, 4760]]",[contextual],8205,"[continuous,periodic]",wave,frequency,wave,1,0,1,1,1,1
3,A-4,SMAP,"[[4550, 4660]]",[contextual],8080,"[continuous,periodic]",wave,frequency,wave,1,0,1,1,1,1
4,A-5,SMAP,"[[2750, 2800]]",[point],4693,[continuous],magnitude,point,magnitude,1,1,1,0,1,1


In [5]:
mission = ['SMAP', 'MSL']

idx_mission = 0

mission_df = df[df['spacecraft']== mission[idx_mission]]

In [85]:
indice = 21
l_w = 250
npersegFFT = 20
"""for l_w in l_ws:
    print(l_w)
    for npersegFFT in nfft:
        print(npersegFFT)"""
for indice in range(0,len(df)):
    actual = df.iloc[indice]['chan_id']
    test=np.load(path+'data/test/'+actual+'.npy')

    print("##################################################")
    print("Mission: ", df.iloc[indice]['spacecraft'])
    print("Channel: ", actual)
    print("Labeled Anomalies: ", df.iloc[indice]['anomaly_sequences'])
    print("Num Values: ", df.iloc[indice]['num_values']-l_w)
    print("Shape: ", test.shape)

    x_test, y_test = shape_data(test,train=False)
    Label = -1 + np.zeros(x_test.shape[0])

    index = ast.literal_eval(df.iloc[indice]['anomaly_sequences'])
    for j in range(0,len(index)):
        Label[index[j][0]-l_w-1:index[j][1]+1] = 1

    # LSTM
    model = Sequential()
    model.add(LSTM(80, input_shape=(None, x_test.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(80,return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(n_p))
    model.add(Activation('linear'))

    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #'models/lw_'+str(l_w)+'/'+actual+'.h5'
    model.load_weights('models/lw_'+str(l_w)+'/'+actual.split('-')[0]+'-'+actual.split('-')[1]+'.h5')

    y_pred = model.predict(x_test)
    errLSTM = y_test - y_pred

    window_size = 30
    smoothing_perc= 0.05
    lstm_batch_size= 64
    batch_size= 70

    smoothing_window = 20
    print(smoothing_window)

    moving_averages = []
    spec_chan = []
    e_s = pd.DataFrame(errLSTM).ewm(span=smoothing_window).mean().values.flatten()


    index_w = 0
    while index_w < len(x_test):

        this_window = x_test[index_w,:,0]

        #Moving average
        window_average = sum(this_window) / l_w
        moving_averages.append(window_average)
        index_w += 1

        f, t, Sxx = signal.spectrogram(this_window,mode='magnitude',nperseg=npersegFFT)
        tnn = np.zeros((t.shape[0],))
        for i in range(0,t.shape[0]):
            tnn[i] = abs(sum(Sxx[:,i]))
        spec_chan.append(stdev(tnn))

    moving_averages_arr = y_test -np.reshape(np.array((moving_averages)),(len(moving_averages),1))
    spec_chan_arr = np.reshape(np.array((spec_chan)),(len(spec_chan),1))
    #errLSTM = np.reshape(errLSTM,(errLSTM.shape[0],1))
    errLSTM = np.reshape(e_s,(e_s.shape[0],1))
    features_arr = np.concatenate((errLSTM, spec_chan_arr, moving_averages_arr),axis=1)
    
    """

    trace = [go.Scatter(y= y_test[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
    layout = go.Layout( title='Real'+str(errLSTM.shape))
    fig = go.Figure(data=trace, layout=layout)
    fig.show()

    trace = [go.Scatter(y= errLSTM[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
    layout = go.Layout( title='LSTM'+str(errLSTM.shape))
    fig = go.Figure(data=trace, layout=layout)
    fig.show()

    trace = [go.Scatter(y= e_s, mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
    layout = go.Layout( title='LSTM_Sm'+str(e_s.shape))
    fig = go.Figure(data=trace, layout=layout)
    fig.show()

    trace = [go.Scatter(y= moving_averages_arr[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
    layout = go.Layout( title='MA'+str(moving_averages_arr.shape))
    fig = go.Figure(data=trace, layout=layout)
    fig.show()

    trace = [go.Scatter(y= spec_chan_arr[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
    layout = go.Layout( title='FFT')
    fig = go.Figure(data=trace, layout=layout)
    fig.show()"""

    np.save('data/features_emaw_ma/'+str(actual)+'.npy',features_arr)

##################################################
Mission:  SMAP
Channel:  A-1
Labeled Anomalies:  [[4690, 4774]]
Num Values:  8390
Shape:  (8640, 25)
20
##################################################
Mission:  SMAP
Channel:  A-2
Labeled Anomalies:  [[4450, 4560]]
Num Values:  7664
Shape:  (7914, 25)
20
##################################################
Mission:  SMAP
Channel:  A-3
Labeled Anomalies:  [[4575, 4760]]
Num Values:  7955
Shape:  (8205, 25)
20
##################################################
Mission:  SMAP
Channel:  A-4
Labeled Anomalies:  [[4550, 4660]]
Num Values:  7830
Shape:  (8080, 25)
20
##################################################
Mission:  SMAP
Channel:  A-5
Labeled Anomalies:  [[2750, 2800]]
Num Values:  4443
Shape:  (4693, 25)
20
##################################################
Mission:  SMAP
Channel:  A-6
Labeled Anomalies:  [[1890, 1930]]
Num Values:  4203
Shape:  (4453, 25)
20
##################################################
Mission:  SMAP
Chan

20
##################################################
Mission:  SMAP
Channel:  P-2
Labeled Anomalies:  [[5300, 6420]]
Num Values:  7959
Shape:  (8209, 25)
20
##################################################
Mission:  SMAP
Channel:  P-3
Labeled Anomalies:  [[5401, 6736]]
Num Values:  8243
Shape:  (8493, 25)
20
##################################################
Mission:  SMAP
Channel:  P-4-1
Labeled Anomalies:  [[950,1080]]
Num Values:  1365
Shape:  (1615, 25)
20
##################################################
Mission:  SMAP
Channel:  P-4-2
Labeled Anomalies:  [[535,735]]
Num Values:  1695
Shape:  (1945, 25)
20
##################################################
Mission:  SMAP
Channel:  P-4-3
Labeled Anomalies:  [[1210,1320]]
Num Values:  3973
Shape:  (4223, 25)
20
##################################################
Mission:  SMAP
Channel:  P-7
Labeled Anomalies:  [[4950, 6600]]
Num Values:  7821
Shape:  (8071, 25)
20
##################################################
Mission:  SMAP
C

In [78]:
import plotly.graph_objects as go

trace = [go.Scatter(y= y_test[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
layout = go.Layout( title='Real'+str(errLSTM.shape))
fig = go.Figure(data=trace, layout=layout)
fig.show()

trace = [go.Scatter(y= errLSTM[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
layout = go.Layout( title='LSTM'+str(errLSTM.shape))
fig = go.Figure(data=trace, layout=layout)
fig.show()

trace = [go.Scatter(y= e_s, mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
layout = go.Layout( title='LSTM_Sm'+str(e_s.shape))
fig = go.Figure(data=trace, layout=layout)
fig.show()

trace = [go.Scatter(y= moving_averages_arr[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
layout = go.Layout( title='MA'+str(moving_averages_arr.shape))
fig = go.Figure(data=trace, layout=layout)
fig.show()

trace = [go.Scatter(y= spec_chan_arr[:,0], mode='lines',name='actual'),go.Scatter(y= Label, mode='lines',name='actual')]
layout = go.Layout( title='FFT')
fig = go.Figure(data=trace, layout=layout)
fig.show()